Just starting to think about how I can do this. 

# Parsing Statistics and Data

In [1]:
#importing driver for webpage
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
#importing parser
from bs4 import BeautifulSoup
#importing pandas for dataframes
import pandas as pd
#delay so java script has time to load
import time
#numpy is always useful
import numpy as np
#used for pandas
from functools import reduce


In [2]:
currTeamAbbriev = "BUF"
#options used later to simplify arguments
# Set up headless browser which stops any GUI from apearing
options = Options()
options.add_argument('--headless')
#stops Graphics processing unit from rendering the web content
options.add_argument('--disable-gpu')

#sets up remote control interface to instruct the behavior of web browsers
#basically sets up functionallity to scrape web
driver = webdriver.Chrome(options=options)

url = f'https://www.pro-football-reference.com/teams/buf/2020.htm'
driver.get(url)

headersArr = []

#reads through page
soup = BeautifulSoup(driver.page_source, 'html.parser')
table = soup.find(id = "games")
#strips *table header* elements
headers = [th.text.strip() for th in table.find_all('th')]
headersArr.append(headers)
#strips all rows for each team
rows = []
boxscore_links = []
for row in table.find_all('tr')[1:]:
    # finds each boxscore link on a teams season page
    boxscore_cell = row.find('td', {'data-stat': 'boxscore_word'})
    if boxscore_cell and boxscore_cell.find('a'):
        #
        link = boxscore_cell.find('a')['href']
        #makes the full link then appends it to list
        full_link = f"https://www.pro-football-reference.com{link}"
        boxscore_links.append(full_link)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
def getGameInfo(currID, driver):
    headersArr = []
    #reads through page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #get the passed in id in link form
    table = soup.find(id = currID)
    #strips *table header* elements
    headers = [th.text.strip() for th in table.find_all('th')]

    headersArr.append(headers)
    columnArr = []
    rows = []
    #strips all rows for each team
    for row in table.find_all('tr')[1:]:
        #td holds info, tr is the stucture
        for td in row.find_all('td'):
            columnArr.append(td.text.strip())
    if columnArr:
        print(columnArr)
        rows.append(columnArr)

    dfStat = pd.DataFrame(rows, columns=np.transpose(headers))
    return dfStat

def returnEndScores(currID, driver):
    homeTeamBool = False
    headersArr = []
    #reads through page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #get the passed in id in link form
    table = soup.find(id = currID)

    
    thead = table.find("thead")
    homeTeam = thead.find("th", {"data-stat": "home_team_score"}).text.strip()
    if homeTeam == currTeamAbbriev:
        homeTeamBool = True
        print(f"{currTeamAbbriev} is home")
    else:
        print(f"{currTeamAbbriev} is away")

    tbody = table.find("tbody")
    visitor_team = tbody.find_all("td", {"data-stat": "vis_team_score"})[-1].text.strip()
    home_team = tbody.find_all("td", {"data-stat": "home_team_score"})[-1].text.strip()

    return visitor_team, home_team, homeTeamBool


def getTeamStats(currID, driver):  
    defensiveHeaders = []
    headersArr = []
    #reads through page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #get the passed in id in link form
    table = soup.find(id = currID)

    #strips *table header* elements
    headers = [th.text.strip() for th in table.find_all('th')]
    
    headersArr.append(headers)
    columnArr = []
    rows = []
    #strips all rows for each team
    headers = []
    visitor_vals = []
    home_vals = []



    # thead = table.find("thead")
    # visitor_team = thead.find("th", {"data-stat": "vis_stat"}).text.strip()
    # home_team = thead.find("th", {"data-stat": "home_stat"}).text.strip()


    for row in table.find_all('tr'):
        stat_name = row.find('th').text.strip()
        cols = row.find_all('td')
        if len(cols) == 2:
            headers.append(stat_name)
            visitor_vals.append(cols[0].text.strip())
            home_vals.append(cols[1].text.strip())

    
    visitorScore, HomeScore, homeTeamBool = returnEndScores("scoring", driver)
    
    defensiveHeaders = headers + [f"Opp_{h}" for h in headers]  # now double headers

    if homeTeamBool:
        home_vals = home_vals + visitor_vals
        print(home_vals)
        print(defensiveHeaders)
        dfStat = pd.DataFrame([home_vals],
                        index=[currTeamAbbriev],
                        columns=defensiveHeaders)
        dfStat["HomeTeam"] = True
        dfStat[f"{currTeamAbbriev} Score"] = HomeScore
        dfStat[f"Opp Score"] = visitorScore
    else:
        visitor_vals = visitor_vals + home_vals
        dfStat = pd.DataFrame([visitor_vals],
                        index= [currTeamAbbriev],
                        columns=defensiveHeaders)
        dfStat["HomeTeam"] = False
        dfStat[f"{currTeamAbbriev} Score"] = visitorScore
        dfStat[f"Opp Score"] = HomeScore
    return dfStat

In [ ]:
masterInfoDF = pd.DataFrame()
masterStatsDF = pd.DataFrame()


for i in range(len(boxscore_links)):
    driver.get(boxscore_links[i])

    gameInfoDF = getGameInfo("game_info", driver)
    gameStatsDF = getTeamStats("team_stats", driver)

    gameInfoDF.columns = pd.MultiIndex.from_product([[f"Game{i}"], gameInfoDF.columns])
    gameStatsDF.columns = pd.MultiIndex.from_product([[f"Game{i}"], gameStatsDF.columns])

    masterInfoDF = pd.concat([masterInfoDF, gameInfoDF], axis=1)
    masterStatsDF = pd.concat([masterStatsDF, gameStatsDF], axis=1)

masterInfoDF = masterInfoDF.stack()
masterStatsDF = masterStatsDF.stack()

In [ ]:
masterStatsDF.stack()

In [ ]:
defensiveHeaders

In [ ]:
def getDataColumns(currID):
    driver.get(link)
    headersArr = []
    #reads through page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #get the passed in id in link form
    table = soup.find(id = currID)
    
    #strips *table header* elements
    headers = [th.text.strip() for th in table.find_all('th')]
    
    headersArr.append(headers)

    rows = []
    #strips all rows for each team
    for row in table.find_all('tr')[1:]:
        #td holds info, tr is the stucture
        cols = [td.text.strip() for td in row.find_all('td')]
        if cols:
            print(cols)
            rows.append(cols)
    dfStat = pd.DataFrame(rows, columns=headers)

    return dfStat

for link in boxscore_links:
    all_stat_dfs = []
    #all_stat_dfs.append(getDataRows("game_info"))
    all_stat_dfs.append(getDataRows("team_stats"))
    all_stat_dfs.append(getDataColumns("kicking"))
    



In [ ]:
boxscore_links

In [ ]:
# Parse the table
